In [21]:
// Kotlin notebook for Tutorial 8: Optimal Load Balancing
%use lets-plot

import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.util.matrix.*
import jline.util.Maths
import kotlin.math.*
import java.util.function.Function

// Set verbosity to standard
GlobalConstants.setVerbose(VerboseLevel.STD)

In [22]:
// Block 1: Create network and nodes
val model = Network("LoadBalCQN")

val delay = Delay(model, "Think")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)

In [23]:
// Block 2: Create closed class with 16 jobs
val cclass = ClosedClass(model, "Job1", 16, delay)

// Set service rates
delay.setService(cclass, Exp(1.0))   // Think time
queue1.setService(cclass, Exp(0.75)) // Faster server
queue2.setService(cclass, Exp(0.50)) // Slower server

In [24]:
// Block 3: Define initial topology
val P = model.initRoutingMatrix()
// IMPORTANT: Set up the return paths from queues to delay first
P.set(cclass, cclass, queue1, delay, 1.0)  // From queue1 back to delay
P.set(cclass, cclass, queue2, delay, 1.0)  // From queue2 back to delay
model.link(P)

In [25]:
// Define the routing model function
fun routingModel(p: Double): Double {
    // Update routing probabilities from delay to queues
    P.set(cclass, cclass, delay, queue1, p)      // Probability to queue1
    P.set(cclass, cclass, delay, queue2, 1 - p)  // Probability to queue2
    model.relink(P)
    
    // Solve and return system response time
    val solver = MVA(model)
    solver.options.method = "exact"
    solver.options.verbose = VerboseLevel.SILENT
    val R = solver.getAvgSysRespT()
    return R.get(0,0)  // Get the first element from the matrix
}

In [26]:
// Test a few points to see the behavior
println("Response time analysis:")
for (p in arrayOf(0.1, 0.3, 0.5, 0.7, 0.9)) {
    val respTime = routingModel(p)
    println("p = $p: Response Time = %.4f".format(respTime))
}

Response time analysis:
p = 0.1: Response Time = 28.8000
p = 0.3: Response Time = 22.4000
p = 0.5: Response Time = 16.0134
p = 0.7: Response Time = 14.9416
p = 0.9: Response Time = 19.2000


In [27]:
// Run optimization using jline.util.Maths.goldenSectionSearch
val objectiveFunction = Function<Double, Double> { p -> routingModel(p) }
val result = Maths.goldenSectionSearch(objectiveFunction, 0.0, 1.0)

val p_opt = result[0]
val min_resp_time = result[1]

println("\nOptimization Results:")
println("Optimal routing probability p*: %.6f".format(p_opt))
println("Minimal response time: %.6f".format(min_resp_time))


Optimization Results:
Optimal routing probability p*: 0.610490
Minimal response time: 13.638237


In [33]:
// Create an inline plot with proper graphics using lets-plot
// Generate data points for the plot
val pValues = (1..99).map { it / 100.0 }
val responseValues = pValues.map { p -> routingModel(p) }

// Create data map for lets-plot
val data = mapOf(
    "p" to pValues,
    "response_time" to responseValues
)

// Create the main line plot
val plot = letsPlot(data) { x = "p"; y = "response_time" } +
    geomLine(color = "blue", size = 1.5) +
    geomPoint(x = p_opt, y = min_resp_time, color = "red", size = 4) +
    labs(
        title = "Response Time vs Routing Probability - Optimal p* = %.3f".format(p_opt),
        x = "Routing Probability p (to Queue1)",
        y = "System Response Time"
    )

// Display the plot
plot

<path d="M24.63812677677562 14.590909090909008 L24.63812677677562 14.590909090909008 L29.666315914893097 19.766778877833588 L34.69450505301057 24.942648664757996 L39.72269419112804 30.118518451682462 L44.750883329245525 35.2943882386071 L49.779072467362994 40.47025802553145 L54.80726160548048 45.646127812455745 L59.835450743597946 50.82199759938044 L64.86363988171541 55.99786738630468 L69.8918290198329 61.17373717322897 L74.92001815795037 66.34960696015293 L79.94820729606784 71.52547674707557 L84.97639643418532 76.70134653399577 L90.0045855723028 81.8772163209099 L95.03277471042026 87.05308610780872 L100.06096384853774 92.22895589467237 L105.08915298665522 97.40482568145632 L110.11734212477268 102.58069546806331 L115.14553126289016 107.75656525428587 L120.17372040100764 112.9324350396904 L125.2019095391251 118.10830482338577 L130.23009867724258 123.28417460357167 L135.25828781536006 128.46004437666886 L140.28647695347752 133.6359141356608 L145.314666091595 138.81178386697974 L150.34285522971248 143.9876535447168 L155.37104436782997 149.16352311997082 L160.39923350594745 154.33939250144732 L165.42742264406488 159.515261520433 L170.45561178218236 164.69112986809847 L175.48380092029984 169.86699698414776 L180.51199005841733 175.04286186052826 L185.5401791965348 180.2187226978204 L190.5683683346523 185.39457630772245 L195.59655747276972 190.57041708058483 L200.6247466108872 195.7462352122277 L205.65293574900468 200.9220136766524 L210.68112488712217 206.09772308775337 L215.70931402523965 211.27331302872386 L220.73750316335713 216.44869750745886 L225.76569230147456 221.62373070842722 L230.79388143959204 226.7981668310572 L235.82207057770952 231.97159404385366 L240.850259715827 237.14332673520408 L245.8784488539445 242.3122313335279 L250.90663799206197 247.47644777388723 L255.9348271301794 252.63294990167253 L260.9630162682969 257.7768629135138 L265.99120540641434 262.90042538423563 L271.01939454453185 267.9914532198168 L276.0475836826493 273.03114836809175 L281.0757728207668 277.9911286761298 L286.10396195888427 282.82969354820165 L291.1321510970018 287.48766194587057 L296.16034023511924 291.88469368104865 L301.18852937323675 295.9177981996154 L306.21671851135414 299.4644521404022 L311.2449076494716 302.3927087244463 L316.2730967875891 304.5790216113156 L321.30128592570657 305.93096947016727 L326.3294750638241 306.4080716503248 L331.35766420194153 306.03242596815994 L336.385853340059 304.8842103531808 L341.4140424781765 303.0838132237511 L346.44223161629395 300.7679946429147 L351.47042075441146 298.0684654435479 L356.4986098925289 295.0980109747434 L361.52679903064643 291.9449076370513 L366.5549881687638 288.6735100849081 L371.5831773068813 285.32812403057915 L376.6113664449988 281.9378561161125 L381.63955558311625 278.52109564831903 L386.66774472123376 275.0890808365425 L391.6959338593512 271.64848316656355 L396.7241229974687 268.2031613496483 L401.7523121355862 264.75529429028796 L406.78050127370363 261.3060847035504 L411.80869041182115 257.85618197982365 L416.8368795499386 254.4059291581891 L421.8650686880561 250.95550348304567 L426.89325782617357 247.5049944769255 L431.92144696429096 244.0544462988875 L436.9496361024085 240.60388019580137 L441.97782524052593 237.15330612359799 L447.00601437864344 233.70272861713931 L452.0342035167609 230.2521496798629 L457.0623926548784 226.8015701679975 L462.09058179299586 223.35099043448253 L467.1187709311133 219.900410619152 L472.14696006923083 216.44983077504708 L477.1751492073483 212.99925092134447 L482.2033383454658 209.54867106462052 L487.23152748358325 206.098091207003 L492.25971662170065 202.64751134913803 L497.28790575981816 199.1969314912089 L502.3160948979356 195.74635163326417 L507.3442840360531 192.29577177531564 L512.3724731741705 188.84519191736644 L517.4006623122881 185.39461205941694 " fill="none" stroke-width="3.3000000000000003" stroke="rgb(0,0,255)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 
 